# 项目：整理Netflix电影演员评分数据

## 分析目标

此数据分析的目的是，整理不同流派影视作品，比如喜剧片、动作片、科幻片中，各演员出演作品的平均IMDB评分，从而挖掘出各个流派中的高评分作品演员。

本实战项目的目的在于练习整理数据，从而得到可供下一步分析的数据。

## 简介

原始数据集记录了截止至2022年7月美国地区可观看的所有Netflix电视剧及电影数据。数据集包含两个数据表：`titles.csv`和`credits.csv`。

`titles.csv`包含电影及电视剧相关信息，包括影视作品ID、标题、类型、描述、流派、IMDB（一个国外的在线评分网站）评分，等等。`credits.csv`包含超过7万名出现在Netflix影视作品的导演及演员信息，包括名字、影视作品ID、人物名、演职员类型（导演/演员）等。

`titles.csv`每列的含义如下：
- id：影视作品ID。
- title：影视作品标题。
- show_type：作品类型，电视节目或电影。
- description：简短描述。
- release_year：发布年份。
- age_certification：适龄认证。
- runtime：每集电视剧或电影的长度。
- genres：流派类型列表。
- production_countries：出品国家列表。
- seasons：如果是电视剧，则是季数。
- imdb_id：IMDB的ID。
- imdb_score：IMDB的评分。
- imdb_votes：IMDB的投票数。
- tmdb_popularity：TMDB的流行度。
- tmdb_score：TMDB的评分。

`credits.csv`每列的含义如下：
- person_ID：演职员ID。
- id：参与的影视作品ID。
- name：姓名。
- character_name：角色姓名。
- role：演职员类型，演员或导演。

## 一.数据评估与清理

In [1]:
import pandas as pd

In [2]:
original_titles=pd.read_csv("./titles.csv")

In [3]:
original_credits=pd.read_csv("./credits.csv")

In [4]:
clean_titles=original_titles.copy()
clean_credits=original_credits.copy()

### 1.乱数据评估与清理

In [5]:
original_titles.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600


### 观察发现genres和production_countries这两列中存在多个值的情况，因此需要拆分

In [6]:
clean_titles["genres"][2]

"['drama', 'action', 'thriller', 'european']"

In [7]:
clean_titles["production_countries"][4]

"['GB', 'US']"

### 发现genres和production_countries是字符串并非列表，因此需要先转换成列表

In [8]:
clean_titles["genres"]=clean_titles["genres"].apply(lambda x:eval(x))

In [9]:
clean_titles["genres"][2]

['drama', 'action', 'thriller', 'european']

In [10]:
clean_titles["production_countries"]=clean_titles["production_countries"].apply(lambda x:eval(x))

In [11]:
clean_titles["production_countries"][4]

['GB', 'US']

### 转换完成后可以调用explode方法来拆分

In [12]:
clean_titles=clean_titles.explode("genres")
clean_titles=clean_titles.explode("production_countries")

In [13]:
clean_titles.sample(10)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
306,tm159901,Miss Congeniality 2: Armed and Fabulous,MOVIE,After her triumph at the Miss United States pa...,2005,PG-13,115,crime,US,NaN,tt0385307,5.1,74637.0,21.495,5.700
3653,tm885805,Debt Collectors,MOVIE,A pair of debt collectors are thrust into an e...,2020,R,97,comedy,US,NaN,tt10933680,5.8,4347.0,15.007,6.100
5346,ts302056,Titletown High,SHOW,In a Georgia town where football rules and win...,2021,TV-14,34,sport,US,1.0,tt15192032,5.5,332.0,2.146,5.600
4371,tm460259,Blood Will Tell,MOVIE,Thriller about a family whose ties are put to ...,2019,NaN,113,drama,AR,NaN,tt5687236,6.1,831.0,2.268,6.000
4635,tm476496,Bhai - Vyakti Ki Valli 2,MOVIE,The second part of the P.L Deshpande biopic wh...,2019,R,130,drama,IN,NaN,tt9680076,7.6,203.0,1.015,8.000
2876,tm429174,Gun City,MOVIE,"Barcelona, ​​Spain, 1921. A tough cop from Mad...",2018,NaN,127,history,FR,NaN,tt7104984,6.3,3895.0,6.610,6.400
4533,tm476922,Wanted - Matloubin,MOVIE,Wanted is a very human comedy about four old p...,2019,NaN,90,comedy,LB,NaN,tt9194246,4.5,615.0,NaN,NaN
4403,tm918258,Dogwashers,MOVIE,When a narco past his prime refuses to pay a d...,2020,NaN,107,drama,AR,NaN,tt13248198,5.8,601.0,29.101,6.313
1307,ts36630,Dinotrux,SHOW,"Half dinosaur, half construction truck, full-o...",2015,TV-Y7,23,family,US,5.0,tt1396212,6.7,465.0,8.914,7.400
3325,tm498034,The Last Blockbuster,MOVIE,A documentary on the last remaining Blockbuste...,2020,NaN,86,documentation,US,NaN,tt8704802,6.7,5448.0,8.213,6.600


In [14]:
original_credits.head()

,person_id,id,name,character,role
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR


### 观察发现credits没有什么结构问题

### 2.脏数据评估与清理

### （1）缺失数据

In [15]:
clean_titles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17818 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    17818 non-null  object 
 1   title                 17817 non-null  object 
 2   type                  17818 non-null  object 
 3   description           17790 non-null  object 
 4   release_year          17818 non-null  int64  
 5   age_certification     10889 non-null  object 
 6   runtime               17818 non-null  int64  
 7   genres                17755 non-null  object 
 8   production_countries  17439 non-null  object 
 9   seasons               6224 non-null   float64
 10  imdb_id               17116 non-null  object 
 11  imdb_score            16976 non-null  float64
 12  imdb_votes            16945 non-null  float64
 13  tmdb_popularity       17663 non-null  float64
 14  tmdb_score            17241 non-null  float64
dtypes: float64(5), int64(2), 

### 在我们关心的数据id、genres和imdb_score中，genres和imdb_score中存在缺失，另外发现release_year应该为日期类型而非整型

In [16]:
clean_titles["release_year"]=pd.to_datetime(clean_titles["release_year"])

In [17]:
clean_titles["release_year"]

0      1970-01-01 00:00:00.000001945
1      1970-01-01 00:00:00.000001976
1      1970-01-01 00:00:00.000001976
2      1970-01-01 00:00:00.000001972
2      1970-01-01 00:00:00.000001972
                    ...             
5847   1970-01-01 00:00:00.000002021
5848   1970-01-01 00:00:00.000002021
5849   1970-01-01 00:00:00.000002021
5849   1970-01-01 00:00:00.000002021
5849   1970-01-01 00:00:00.000002021
Name: release_year, Length: 17818, dtype: datetime64[ns]

In [18]:
clean_titles[clean_titles["genres"].isnull()]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
212,tm255589,One Last Shot,MOVIE,"In this low-budget short film, two best buddie...",1970-01-01 00:00:00.000001998,NaN,30,NaN,NaN,NaN,NaN,NaN,NaN,1.890,5.2
619,tm341561,Like Twenty Impossibles,MOVIE,Occupied Palestine: A serene landscape now poc...,1970-01-01 00:00:00.000002003,NaN,16,NaN,PS,NaN,NaN,NaN,NaN,0.812,6.5
632,ts86241,Le Robe De Mariage Des Cieux,SHOW,It was with much difficulty that Ai Qing was a...,1970-01-01 00:00:00.000002004,TV-MA,63,NaN,NaN,1.0,NaN,NaN,NaN,0.600,NaN
636,tm404676,To and from New York,MOVIE,"While covering a story in New York City, a Sea...",1970-01-01 00:00:00.000002006,NaN,82,NaN,US,NaN,NaN,NaN,NaN,1.401,5.8
637,tm89054,Osuofia in London 2,MOVIE,Osuofia return to his Nigerian village with a ...,1970-01-01 00:00:00.000002004,NaN,72,NaN,XX,NaN,NaN,NaN,NaN,1.091,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5799,tm1040959,The Circle: The Afterparty,MOVIE,Stars of The Circle drop by to discuss Season ...,1970-01-01 00:00:00.000002021,NaN,35,NaN,US,NaN,NaN,NaN,NaN,1.882,10.0
5802,ts302434,Plastic Cup Boyz: Laughing My Mask Off!,SHOW,Comedy collective The Plastic Cup Boyz pour ou...,1970-01-01 00:00:00.000002021,NaN,33,NaN,NaN,1.0,NaN,NaN,NaN,0.683,NaN
5840,tm1216735,Sun of the Soil,MOVIE,"In 14th-century Mali, an ambitious young royal...",1970-01-01 00:00:00.000002022,NaN,26,NaN,NaN,NaN,NaN,NaN,NaN,1.179,7.0
5844,tm1074617,Bling Empire - The Afterparty,MOVIE,"The stars of ""Bling Empire"" discuss the show's...",1970-01-01 00:00:00.000002021,NaN,35,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
clean_titles[(clean_titles["genres"].isnull()) & (~clean_titles["imdb_score"].isnull())]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
1813,ts77824,My Next Guest Needs No Introduction With David...,SHOW,TV legend David Letterman teams up with fascin...,1970-01-01 00:00:00.000002018,TV-MA,50,NaN,US,4.0,tt7829834,7.8,5581.0,8.217,7.6
1939,ts215037,Minecraft: Story Mode,SHOW,"MInecraft: Story Mode is an interactive, anima...",1970-01-01 00:00:00.000002018,TV-PG,52,NaN,US,1.0,tt10498322,5.6,347.0,NaN,NaN
2386,ts74805,A Little Help with Carol Burnett,SHOW,In this unscripted series starring comedy lege...,1970-01-01 00:00:00.000002018,TV-G,24,NaN,US,1.0,tt7204366,6.3,237.0,1.621,6.2
2658,ts265844,#ABtalks,SHOW,#ABtalks is a YouTube interview show hosted by...,1970-01-01 00:00:00.000002018,TV-PG,68,NaN,NaN,1.0,tt12635254,9.6,7.0,NaN,NaN
4274,tm1172010,The Lockdown Plan,MOVIE,NaN,1970-01-01 00:00:00.000002020,NaN,49,NaN,NaN,NaN,tt13079112,6.5,NaN,NaN,NaN
4648,tm1113921,In Vitro,MOVIE,'In Vitro' is an otherworldly rumination on me...,1970-01-01 00:00:00.000002019,NaN,27,NaN,NaN,NaN,tt10545994,7.7,NaN,NaN,NaN


### 观察发现少数数据genres缺失但imdb_score没有缺失，但这部分数据仍然需要删除

In [20]:
clean_titles=clean_titles.dropna(subset=["genres"])

In [21]:
clean_titles[clean_titles["genres"].isnull()]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score


In [22]:
clean_titles[clean_titles["imdb_score"].isnull()]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1970-01-01 00:00:00.000001945,TV-MA,51,documentation,US,1.0,NaN,NaN,NaN,0.600,NaN
75,tm132164,Bill Hicks: Sane Man,MOVIE,Sane Man was filmed before Bill recorded ‘Dang...,1970-01-01 00:00:00.000001989,R,80,comedy,US,NaN,NaN,NaN,NaN,3.377,7.5
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1970-01-01 00:00:00.000001991,TV-G,18,documentation,JP,12.0,NaN,NaN,NaN,7.730,7.8
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1970-01-01 00:00:00.000001991,TV-G,18,family,JP,12.0,NaN,NaN,NaN,7.730,7.8
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1970-01-01 00:00:00.000001991,TV-G,18,reality,JP,12.0,NaN,NaN,NaN,7.730,7.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5790,tm1094060,My Heroes Were Cowboys,MOVIE,Robin Wiltshire's painful childhood was rescue...,1970-01-01 00:00:00.000002021,PG,23,documentation,US,NaN,tt15084326,NaN,NaN,3.145,7.7
5791,tm1047429,Alan Saldaña: Locked Up,MOVIE,"Mexican comedian Alan Saldaña is back, poking ...",1970-01-01 00:00:00.000002021,NaN,49,comedy,NaN,NaN,NaN,NaN,NaN,6.670,6.0
5810,tm1225897,Social Man,MOVIE,Two competitive social media Influencers go he...,1970-01-01 00:00:00.000002021,NaN,96,comedy,NaN,NaN,tt20198164,NaN,NaN,NaN,NaN
5810,tm1225897,Social Man,MOVIE,Two competitive social media Influencers go he...,1970-01-01 00:00:00.000002021,NaN,96,drama,NaN,NaN,tt20198164,NaN,NaN,NaN,NaN


In [23]:
clean_titles=clean_titles.dropna(subset=["imdb_score"])

In [24]:
clean_titles[clean_titles["imdb_score"].isnull()]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score


### 成功将titles中genres和imdb_score缺失的部分删除

### titels中我们关心的数据是genres和imdb_score，而credits中我们关心的数据是person_ID、id和role

In [25]:
clean_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77801 entries, 0 to 77800
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  77801 non-null  int64 
 1   id         77801 non-null  object
 2   name       77801 non-null  object
 3   character  68029 non-null  object
 4   role       77801 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.0+ MB


### 可见crdits中只有部分character缺失，其余数据都是完整无缺的，但person_id应该为字符串类型而非整型

In [26]:
clean_credits["person_id"]=clean_credits["person_id"].astype(str)

In [27]:
clean_credits["person_id"]

0           3748
1          14658
2           7064
3           3739
4          48933
          ...   
77796     736339
77797     399499
77798     373198
77799     378132
77800    1950416
Name: person_id, Length: 77801, dtype: object

### （2）重复数据

In [28]:
print(clean_titles.duplicated().sum())

0


In [29]:
clean_titles.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1970-01-01 00:00:00.000001976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1970-01-01 00:00:00.000001976,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1970-01-01 00:00:00.000001972,R,109,drama,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1970-01-01 00:00:00.000001972,R,109,action,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1970-01-01 00:00:00.000001972,R,109,thriller,US,NaN,tt0068473,7.7,107673.0,10.010,7.300


In [30]:
print(clean_credits.duplicated().sum())

0


### 发现clean_titles和clean_credits中都没有重复数据

### （3）不一致数据

### 对于clean_titles，可能出现不一致数据的有production_countries和genres

In [31]:
pd.set_option("display.max_rows",None)

In [32]:
clean_titles["production_countries"].value_counts().sort_index()

production_countries
AE           52
AF            1
AL            6
AO            3
AR          150
AT           28
AU          217
BD            3
BE          148
BF            2
BG           31
BR          221
BS            3
BT            2
BY            2
CA          608
CD            7
CH           33
CL           35
CM            3
CN          295
CO           94
CU            1
CY            5
CZ           22
DE          383
DK           89
DZ            3
EG           93
ES          637
FI           18
FO            1
FR          720
GB         1068
GE            1
GH            6
GL            3
GR            4
GT            1
HK          102
HR            3
HU           18
ID          149
IE           49
IL           42
IN         1610
IO            4
IQ            5
IR            7
IS           28
IT          224
JO           19
JP         1046
KE            6
KG            2
KH           10
KN            4
KR          637
KW           10
LB           70
LK            1
LT 

### Lebanon和LB指代的应该是同一个国家，因此需要统一

In [33]:
clean_titles=clean_titles.replace({"Lebanon":"LB"})

In [34]:
clean_titles["production_countries"].value_counts().sort_index()

production_countries
AE      52
AF       1
AL       6
AO       3
AR     150
AT      28
AU     217
BD       3
BE     148
BF       2
BG      31
BR     221
BS       3
BT       2
BY       2
CA     608
CD       7
CH      33
CL      35
CM       3
CN     295
CO      94
CU       1
CY       5
CZ      22
DE     383
DK      89
DZ       3
EG      93
ES     637
FI      18
FO       1
FR     720
GB    1068
GE       1
GH       6
GL       3
GR       4
GT       1
HK     102
HR       3
HU      18
ID     149
IE      49
IL      42
IN    1610
IO       4
IQ       5
IR       7
IS      28
IT     224
JO      19
JP    1046
KE       6
KG       2
KH      10
KN       4
KR     637
KW      10
LB      71
LK       1
LT       7
LU      27
MA      15
MC       4
MT       8
MU       5
MW       2
MX     264
MY      30
NG     131
NL     102
NO      68
NP       1
NZ      27
PE      26
PH     192
PK       9
PL     126
PR       9
PS      32
PT      14
PY       3
QA      24
RO      25
RS       2
RU      41
SA      28
SE      81


In [35]:
clean_titles["genres"].value_counts()

genres
drama            3357
comedy           2419
thriller         1446
action           1339
romance          1080
crime            1066
documentation     981
family            769
animation         732
fantasy           727
european          679
scifi             647
horror            438
history           336
music             266
reality           226
war               221
sport             188
western            53
Name: count, dtype: int64

In [36]:
clean_titles.query('genres==""')

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score


### 对于clean_crdits，可能存在不一致的数据有role

In [37]:
clean_credits["role"].value_counts().sort_index()

role
ACTOR       73251
DIRECTOR     4550
Name: count, dtype: int64

### 由此可见role只有导演和演员两种，但由于我们的目标是找出评分最高的演员，因此可以将导演的数据删除

In [38]:
clean_credits=clean_credits[clean_credits["role"]=="ACTOR"]

In [39]:
clean_credits[clean_credits["role"]=="DIRECTOR"]

,person_id,id,name,character,role


### (4）无效数据

In [40]:
clean_titles.describe()

,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
count,16970,16970.000000,5954.000000,16970.000000,1.694100e+04,16842.000000,16515.000000
mean,1970-01-01 00:00:00.000002015,80.912552,2.455492,6.514207,3.281655e+04,29.396307,6.846933
min,1970-01-01 00:00:00.000001954,0.000000,1.000000,1.500000,5.000000e+00,0.600000,1.000000
25%,1970-01-01 00:00:00.000002015,45.000000,1.000000,5.800000,7.800000e+02,4.070000,6.200000
50%,1970-01-01 00:00:00.000002018,90.000000,2.000000,6.600000,3.508000e+03,10.195000,6.900000
75%,1970-01-01 00:00:00.000002020,107.000000,3.000000,7.300000,1.697800e+04,23.639000,7.500000
max,1970-01-01 00:00:00.000002022,225.000000,42.000000,9.500000,2.294231e+06,2274.044000,10.000000
std,NaN,39.596172,2.869428,1.131095,1.141492e+05,93.178235,1.078831


In [41]:
clean_credits.describe()

,person_id,id,name,character,role
count,73251,73251,73251,68029,73251
unique,51544,5340,51308,47274,1
top,38636,tm32982,Boman Irani,Self,ACTOR
freq,25,207,25,1950,73251


### 由此可见clean_credits和clean_titles中没有无效数据

### （5）保存数据

In [42]:
clean_titles.to_csv("./titles.cleaned.csv",index=False)
clean_credits.to_csv("./credits_cleaned.csv",index=False)

## 二.数据整理

### 将celan_titles和clean_credits按id进行合并

In [43]:
titles_with_credits=clean_titles.merge(clean_credits,on="id")
titles_with_credits.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,person_id,name,character,role
0,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1970-01-01 00:00:00.000001976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,3748,Robert De Niro,Travis Bickle,ACTOR
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1970-01-01 00:00:00.000001976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,14658,Jodie Foster,Iris Steensma,ACTOR
2,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1970-01-01 00:00:00.000001976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,7064,Albert Brooks,Tom,ACTOR
3,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1970-01-01 00:00:00.000001976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,3739,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1970-01-01 00:00:00.000001976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,48933,Cybill Shepherd,Betsy,ACTOR


### 我们需要的数据有person_id、genres和imdb_scores，所以将其提取出来

In [46]:
person_id_genres_with_imdb_score=titles_with_credits[["person_id","genres","imdb_score"]]
person_id_genres_with_imdb_score.head()

,person_id,genres,imdb_score
0,3748,drama,8.2
1,14658,drama,8.2
2,7064,drama,8.2
3,3739,drama,8.2
4,48933,drama,8.2


### 为了找到各个流派里平均分最高的演员，我们需要先按照流派分组，再按照演员分组，然后求平均分

In [47]:
genres_person_id_with_mean=person_id_genres_with_imdb_score.groupby(["genres","person_id"])["imdb_score"].mean()
genres_person_id_with_mean.head()

genres  person_id
action  1000         6.866667
        100007       7.000000
        100013       6.400000
        100019       6.500000
        100020       6.500000
Name: imdb_score, dtype: float64

### 然后可以将其转换为一个DataFrame

In [54]:
genres_person_id_with_mean_df=genres_person_id_with_mean.reset_index()
genres_person_id_with_mean_df.head()

,genres,person_id,imdb_score
0,action,1000,6.866667
1,action,100007,7.000000
2,action,100013,6.400000
3,action,100019,6.500000
4,action,100020,6.500000


### 这样我们就得到了各个流派各个演员的平均分，然后再求最大值

In [57]:
genres_with_max=genres_person_id_with_mean_df.groupby("genres")["imdb_score"].max()
genres_with_max

genres
action           9.3
animation        9.3
comedy           9.2
crime            9.5
documentation    9.1
drama            9.5
european         8.9
family           9.3
fantasy          9.3
history          9.1
horror           9.0
music            8.8
reality          8.9
romance          9.2
scifi            9.3
sport            9.1
thriller         9.5
war              8.8
western          8.9
Name: imdb_score, dtype: float64

### 然后再将这个得到的各个流派的最高分和带有person_id的DataFrame进行连接

In [58]:
genres_person_id_with_max=pd.merge(genres_with_max,genres_person_id_with_mean_df,on=["genres","imdb_score"])
genres_person_id_with_max

,genres,imdb_score,person_id
0,action,9.3,12790
1,action,9.3,1303
2,action,9.3,21033
3,action,9.3,336830
4,action,9.3,86591
5,animation,9.3,1303
6,animation,9.3,21033
7,animation,9.3,28024
8,animation,9.3,336830
9,animation,9.3,86591


### 得到这个数据后，我们还需要知道每个person_id对应的演员的姓名，所以可以用最开始的那个DataFrame将person_id和name提取出来，但由于一个演员可能出演多部电影，会出现重复数据，所以需要清理重复数据

In [60]:
person_id_with_name=titles_with_credits[["person_id","name"]].drop_duplicates()
person_id_with_name.head()

,person_id,name
0,3748,Robert De Niro
1,14658,Jodie Foster
2,7064,Albert Brooks
3,3739,Harvey Keitel
4,48933,Cybill Shepherd


In [62]:
transition1=pd.merge(person_id_with_name,genres_person_id_with_max,on="person_id")
transition1.head()

,person_id,name,genres,imdb_score
0,22311,Koichi Yamadera,western,8.9
1,1652,Lukas Haas,music,8.8
2,1641,Leonardo DiCaprio,music,8.8
3,28180,Unsho Ishizuka,western,8.9
4,28166,Megumi Hayashibara,western,8.9


### 观察发现genres都是乱序的，因此需要进行分组

In [64]:
transition2=transition1.sort_values("genres")
transition2.head()

,person_id,name,genres,imdb_score
42,12790,Olivia Hack,action,9.3
37,86591,Cricket Leigh,action,9.3
6,336830,André Sogliuzzo,action,9.3
23,21033,Zach Tyler,action,9.3
32,1303,Jessie Flower,action,9.3


### 又发现索引是混乱的，因此还需要重新设置索引

In [66]:
transition3=transition2.reset_index()
transition3.head()

,index,person_id,name,genres,imdb_score
0,42,12790,Olivia Hack,action,9.3
1,37,86591,Cricket Leigh,action,9.3
2,6,336830,André Sogliuzzo,action,9.3
3,23,21033,Zach Tyler,action,9.3
4,32,1303,Jessie Flower,action,9.3


### 最后删除多出来的index这一列

In [67]:
final=transition3.drop("index",axis=1)
final.head()

,person_id,name,genres,imdb_score
0,12790,Olivia Hack,action,9.3
1,86591,Cricket Leigh,action,9.3
2,336830,André Sogliuzzo,action,9.3
3,21033,Zach Tyler,action,9.3
4,1303,Jessie Flower,action,9.3


### 最后保存一下数据

In [68]:
final.to_csv("./FinalResult.csv",index=False)